# 1、什么是Ajax
基本思想：利用requests来模拟Ajax请求

基本原理：
---
1、发送请求
---
2、解析内容
---
3、渲染网页
---

-发送请求
我们知道JavaScript可以实现页面的各种交互功能，Ajax也不列外，它是由JavaScript实现的，
实际上执行了如下代码：
```
var xmlhttp;
if (window.XMLHttpRequest){
    // code for IE7+, Firefox, Chrome, Opera, Safari
    xmlhttp = new XMLHttpRequest();
} else {  // code for IE6, IE5
    xmlhttp = new ActiveXObject("Microsoft.XMLHTTP");
}
xmlhttp.onreadystatechange=function() {
    if (xmlhttp.readyState == 4 && xmlhttp.status == 200) {
        document.getElementById("myDiv").innerHTML = xmlhttp.responseText;
    }
}
xmlhttp.open("POST", "/ajax/", true);
xmlhttp.send();
```
以上是JavaScript对Ajax最底层的实现，实际上就是新建了XMLHttpRequest对象，然后调用
onreadystatechange属性设置了监听，然后调用open()和send()方法向某个链接（也就是服务器）发送了请求。前面用Python实现请求发送之后，可以得到响应结果，但这里请求的发送变成JavaScript来完成。由于设置了监听，所以当服务器返回相应时，onreadystatrchange对应的方法便会被触发，然后在这个方法里解析响应内容即可
---
-解析内容
得到响应之后， onreadystatechange属性对应的方法便会被触发，此时利用xmlhttp的responseText属性便可取到响应内容。这类似于Python中利用requests向服务器发起请求，然后得到响应的过程。那么返回内容可能是HTML，可能是JSON，接下来只需要在方法中用JavaScript进一步处理即可。比如，如果是JSON的话，可以进行解析和转化。
---
-渲染网页
JavaScript有改变网页内容的能力，解析完响应内容之后，就可以调用JavaScript来针对解析完的内容对网页进行下一步处理了。比如，通过document.getElementById().innerHTML这样的操作，便可以对某个元素内的源代码进行更改，这样网页显示的内容就改变了，这样的操作也被称作DOM操作，即对Document网页文档进行操作，如更改、删除等。
---

上例中，document.getElementById("myDiv").innerHTML=xmlhttp.responseText 便将 ID 为 myDiv 的节点内部的 HTML 代码更改为服务器返回的内容，这样 myDiv 元素内部便会呈现出服务器返回的新数据，网页的部分内容看上去就更新了 。

# 2、Ajax分析方法

---
1、查看请求
---
Ajax有其特殊的请求类型，它叫做 xhr （在检查的network的type里面可以找到），点进去发现其中有一项为 X-Requested-With: XMLHttpRequest ， 这就标记了此请求是Ajax请求。
点击Preview可以看到响应的内容，是JSON格式的。返回的结果就是用来渲染网页所使用的数据。JavaScript接收到这些数据之后，再执行相应的渲染方法，整个页面就渲染出来了。
---
2、过滤请求
---
利用Chrome开发者工具的筛选功能筛选出所有的Ajax请求。在请求的上方有一层筛选栏，直接点击XHR，此时在下方显示的所有请求便都是Ajax请求了。
接下来,不断滑动页面，看到底部有新的内容出现，Ajax请求也不断出现，这样就可以捕获到所有的Ajax请求了。

# 3、Ajax结果提取
---
1、分析请求
---
打开Ajax的XHR过滤器，然后一直滑动页面以加载新的微博内容。可以看到，会不断有Ajax请求发出。
选定其中一个请求，分析它的参数信息。点击该请求，进入详情页面。
发现是一个GET类型的请求，得到请求链接、请求参数。
分析出网页改变时URL相应改变的规律。
---
2、分析响应
---
观察请求的响应内容。
内容是JSON格式的，分析出重要内容
---
3、实战演练
---
用程序模拟Ajax请求爬取崔庆才微博

In [5]:
from urllib.parse import urlencode
import requests
base_url = 'https://m.weibo.cn/api/container/getIndex?'

headers = {
    'Host': 'm.weibo.cn',
    'Referer': 'https://m.weibo.cn/u/2830678474',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

def get_page(page):
    params = {
        'type': 'uid',
        'value': '2830678474',
        'containerid': '1076032830678474',
        'page': page
    }
    url = base_url + urlencode(params)
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200: 
            return response.json()
    except requests.ConnectionError as e: 
        print('Error', e.args)

In [8]:
# 定义一个解析方法，从结果中提取想要的信息。
from pyquery import PyQuery as pq 
def parse_page(json):
    if json:
        items = json.get('data').get('cards')
        for item in items:
            item = item.get('mblog')
            weibo = {}
            try:
                weibo['id'] = item.get('id')
                weibo['text'] = pq(item.get('text')).text()
                weibo['attitudes'] = item.get('attitudes_count')
                weibo['comments'] = item.get('comments_count')
                weibo['reposts'] = item.get('reposts_count')
                yield weibo
            except Exception as e:
                continue

In [9]:
# 最后遍历page
if __name__ == '__main__':
    for page in range(1, 11):
        json = get_page(page)
        results = parse_page(json)
        for result in results:
            print(result)

{'id': '4362348917011512', 'text': '@长泽牙妹', 'attitudes': 0, 'comments': 1, 'reposts': 0}
{'id': '4362348561001001', 'text': '@长泽牙妹', 'attitudes': 0, 'comments': 0, 'reposts': 0}
{'id': '4362235948170191', 'text': '其实挺喜欢某天晚上什么都不管，静静看一会书的时候。', 'attitudes': 2, 'comments': 2, 'reposts': 0}
{'id': '4362235214079741', 'text': '我在大麦发现了一个很棒的演出『李荣浩 【年少有为】巡回演唱会-北京站』，你们也赶快过来看看吧!#去现场为所爱#（通过@大麦网 发布）网页链接 @长泽牙妹', 'attitudes': 1, 'comments': 0, 'reposts': 0}
{'id': '4362227873988237', 'text': '有时候我在想，很多时候我做的一些事到底有没有用，是不是在做无用功。\n其实可以这么思考，现在回想一下，当时我们初中高中学的东西，在现在看来有没有用呢？你可能说现在一点用都没有，全都忘干净了。然而，假若在那个时候我们不知道这些你已经遗忘的知识，我们可以走到今天这一步吗？显然，不能。当 ...全文', 'attitudes': 15, 'comments': 6, 'reposts': 0}
{'id': '4361291709287279', 'text': '昨天小马给我俩一人买了一双拖鞋，安德玛的，一双 279，我心想卧槽这么这么贵，不就是个拖鞋吗？我的拖鞋都二三十块钱，后来一穿，真舒服啊，真香！\n今天小马给我买的 Airpods 到了，本来没用过这玩意，心想就是俩无线耳机啊，后来自动连接、自动检测、自动暂停、自动播放，苹果还是很用心了，卧槽又真 ...全文', 'attitudes': 18, 'comments': 10, 'reposts': 0}
{'id': '4361125753184692', 'text': '司机：？？？\n恍恍惚惚\n不得不说捏完 jiojio 真爽啊 北京·颐润堂专业养生(五道

{'id': '4339029518992449', 'text': '今天我升级为VIP5了', 'attitudes': 3, 'comments': 0, 'reposts': 0}
{'id': '4339018663273602', 'text': '2019，继续前行！ 2019，继续前行！', 'attitudes': 13, 'comments': 0, 'reposts': 2}
{'id': '4338970181900714', 'text': '看吃个螃蟹把你乐呵的，是不是傻？', 'attitudes': 9, 'comments': 6, 'reposts': 0}
{'id': '4338859858547792', 'text': '防抖不错', 'attitudes': 1, 'comments': 0, 'reposts': 0}
{'id': '4338855496611272', 'text': '', 'attitudes': 1, 'comments': 0, 'reposts': 0}
{'id': '4338855178609250', 'text': '//@TONOT时尚视频眼镜:好像真的是一副可以萌化钢筋儿直男的智能眼镜哦', 'attitudes': 0, 'comments': 0, 'reposts': 0}
{'id': '4338852494159499', 'text': '', 'attitudes': 1, 'comments': 0, 'reposts': 0}
{'id': '4338844650186505', 'text': '哇！', 'attitudes': 0, 'comments': 0, 'reposts': 0}
{'id': '4338834327905661', 'text': '真牛逼啊', 'attitudes': 0, 'comments': 1, 'reposts': 0}
{'id': '4338649325512538', 'text': '回复@长泽牙妹: 世界之大，无瓜不有啊！//@长泽牙妹:@崔庆才丨静觅//@粉乳乳:这电影勾起了我的好奇心，想看看了哈哈哈哈哈哈', 'attitudes': 7, 'comments': 1, 'reposts': 2}


In [11]:
# 将结果保存到MongoDB
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['weibo']
collection = db['weibo']

def save_to_mongo(result):
    if collection.insert(result):
        print('Saved to Mongo')